# %% [markdown]
# # 🇩🇰 Denmark Wholesale Electricity Market Analysis
# 
# **Real Wholesale Trading Data Analysis for Quantile Energy Demo**
# 
# This notebook analyzes real Denmark wholesale electricity prices from Nord Pool:
# 
# 1. **Data Loading** - Import your Excel files (2023, 2024, 2025)
# 2. **Market Analysis** - Understand wholesale price patterns
# 3. **Anomaly Detection** - Identify trading opportunities
# 4. **Machine Learning** - Predict wholesale prices
# 5. **Trading Strategies** - Quantify profit potential
# 
# *Perfect for demonstrating systematic energy trading approach to Quantile Energy*

In [1]:
# Cell 1: Setup and Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import os

# ML libraries
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Setup
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

print("🇩🇰 DENMARK WHOLESALE ELECTRICITY ANALYSIS")
print("=" * 50)
print("✅ All libraries imported successfully")
print("⚡ Ready to analyze real wholesale electricity data!")

🇩🇰 DENMARK WHOLESALE ELECTRICITY ANALYSIS
✅ All libraries imported successfully
⚡ Ready to analyze real wholesale electricity data!


In [3]:
# Cell 2: Load Denmark Data - Multiple Excel Files with Pandas
import pandas as pd
import numpy as np

print("🇩🇰 LOADING DENMARK WHOLESALE ELECTRICITY DATA")
print("=" * 55)

# *** STEP 1: UPDATE YOUR FILE PATHS HERE ***
file_paths = [
    "E:\Github projects\Energy_Quant\DK_2023.xlsx",    # Change to your 2023 file name
    "E:\Github projects\Energy_Quant\DK1_2024.xlsx",    # Change to your 2024 file name  
    "E:\Github projects\Energy_Quant\DK1_2025.xlsx"     # Change to your 2025 file name
]

print(f"📁 Loading {len(file_paths)} Excel files:")
for i, path in enumerate(file_paths, 1):
    print(f"   {i}. {path}")

all_dataframes = []

# Load each Excel file
for i, file_path in enumerate(file_paths, 1):
    print(f"\n📊 Processing file {i}/{len(file_paths)}: {file_path}")
    
    try:
        # Read Excel file
        df = pd.read_excel(file_path)
        
        print(f"   ✅ Loaded successfully!")
        print(f"   📊 Shape: {df.shape}")
        print(f"   📋 Columns: {list(df.columns)}")
        
        # Show sample data from first file
        if i == 1:
            print(f"\n📋 Sample from first file:")
            print(df.head(3))
        
        all_dataframes.append(df)
        
    except FileNotFoundError:
        print(f"   ❌ File not found: {file_path}")
        print(f"   💡 Please check the file name and path")
        
    except Exception as e:
        print(f"   ❌ Error loading {file_path}: {e}")

# Check if we loaded any files
if not all_dataframes:
    print(f"\n❌ No files loaded successfully!")
    print(f"💡 Please update the file_paths list above with your actual file names")
    df_wholesale = None
else:
    print(f"\n✅ Loaded {len(all_dataframes)} files successfully!")
    
    # *** STEP 2: SPECIFY COLUMN NAMES ***
    print(f"\n🔧 PROCESSING AND COMBINING DATA:")
    
    # Look at the columns from the first file
    print(f"📋 Columns in first file: {list(all_dataframes[0].columns)}")
    
    # *** UPDATE THESE TO MATCH YOUR COLUMN NAMES ***
    datetime_column = 'Date'        # Your datetime column (Date, Time, Hours, etc.)
    price_column = 'DK1'           # Your price column (DK1, DK2, Denmark, Price, etc.)
    
    print(f"\n🕐 Using datetime column: '{datetime_column}'")
    print(f"💰 Using price column: '{price_column}'")
    
    # Process each dataframe
    processed_dfs = []
    
    for i, df in enumerate(all_dataframes, 1):
        print(f"\n   Processing file {i}...")
        
        try:
            # Check if required columns exist
            if datetime_column not in df.columns:
                print(f"   ❌ Column '{datetime_column}' not found")
                print(f"   Available: {list(df.columns)}")
                continue
                
            if price_column not in df.columns:
                print(f"   ❌ Column '{price_column}' not found") 
                print(f"   Available: {list(df.columns)}")
                continue
            
            # Create clean dataframe
            df_clean = pd.DataFrame()
            df_clean['datetime'] = pd.to_datetime(df[datetime_column])
            df_clean['price_eur_mwh'] = pd.to_numeric(df[price_column], errors='coerce')
            
            # Remove invalid data
            df_clean = df_clean.dropna()
            
            print(f"   ✅ Processed {len(df_clean):,} valid records")
            print(f"   📅 Date range: {df_clean['datetime'].min()} to {df_clean['datetime'].max()}")
            
            processed_dfs.append(df_clean)
            
        except Exception as e:
            print(f"   ❌ Error processing file {i}: {e}")
    
    # Combine all processed dataframes
    if processed_dfs:
        print(f"\n🔗 COMBINING ALL DATA:")
        
        # Concatenate all dataframes
        df_combined = pd.concat(processed_dfs, ignore_index=True)
        
        # Set datetime as index and sort
        df_combined = df_combined.set_index('datetime').sort_index()
        
        # Remove any duplicate timestamps
        original_len = len(df_combined)
        df_combined = df_combined[~df_combined.index.duplicated(keep='first')]
        duplicates_removed = original_len - len(df_combined)
        
        if duplicates_removed > 0:
            print(f"   🧹 Removed {duplicates_removed} duplicate timestamps")
        
        # Final dataset
        df_wholesale = df_combined.copy()
        
        print(f"\n✅ DATA LOADING COMPLETE!")
        print(f"   📊 Total records: {len(df_wholesale):,}")
        print(f"   📅 Full date range: {df_wholesale.index.min()} to {df_wholesale.index.max()}")
        print(f"   📈 Duration: {(df_wholesale.index.max() - df_wholesale.index.min()).days} days")
        print(f"   💰 Price range: €{df_wholesale['price_eur_mwh'].min():.1f} to €{df_wholesale['price_eur_mwh'].max():.1f}/MWh")
        print(f"   📊 Average price: €{df_wholesale['price_eur_mwh'].mean():.1f}/MWh")
        print(f"   📈 Price volatility: €{df_wholesale['price_eur_mwh'].std():.1f}/MWh")
        
        # Show sample of final data
        print(f"\n📋 Sample of combined data:")
        print(df_wholesale.head())
        
        # Check for interesting market events
        negative_prices = (df_wholesale['price_eur_mwh'] < 0).sum()
        extreme_high = (df_wholesale['price_eur_mwh'] > 200).sum()
        
        print(f"\n🎯 Market Events Found:")
        print(f"   • Negative price periods: {negative_prices}")
        print(f"   • Extreme high prices (>€200/MWh): {extreme_high}")
        
        if negative_prices > 0:
            min_price = df_wholesale['price_eur_mwh'].min()
            min_date = df_wholesale[df_wholesale['price_eur_mwh'] == min_price].index[0]
            print(f"   • Lowest price: €{min_price:.1f}/MWh on {min_date}")
        
    else:
        print(f"\n❌ No data could be processed successfully")
        df_wholesale = None

# Summary
if df_wholesale is not None:
    print(f"\n🚀 READY FOR ANALYSIS!")
    print(f"   Real Denmark wholesale electricity data loaded")
    print(f"   {len(df_wholesale):,} periods ready for trading analysis")
else:
    print(f"\n❌ DATA LOADING FAILED")
    print(f"💡 Please check:")
    print(f"   1. File paths are correct")
    print(f"   2. Column names match your data")
    print(f"   3. Files are not open in Excel")
    
    # Show what to update
    print(f"\n🔧 TO FIX:")
    print(f"   1. Update file_paths list with your actual file names")
    print(f"   2. Update datetime_column and price_column variables")
    print(f"   3. Check column names in your Excel files")

🇩🇰 LOADING DENMARK WHOLESALE ELECTRICITY DATA
📁 Loading 3 Excel files:
   1. E:\Github projects\Energy_Quant\DK_2023.xlsx
   2. E:\Github projects\Energy_Quant\DK1_2024.xlsx
   3. E:\Github projects\Energy_Quant\DK1_2025.xlsx

📊 Processing file 1/3: E:\Github projects\Energy_Quant\DK_2023.xlsx
   ✅ Loaded successfully!
   📊 Shape: (8766, 2)
   📋 Columns: ['Energy Prices', 'Unnamed: 1']

📋 Sample from first file:
                               Energy Prices Unnamed: 1
0                     Energy Prices [12.1.D]        NaN
1  01/01/2023 00:00 - 01/01/2024 00:00 (UTC)        NaN
2                                        NaN        NaN

📊 Processing file 2/3: E:\Github projects\Energy_Quant\DK1_2024.xlsx
   ✅ Loaded successfully!
   📊 Shape: (8790, 2)
   📋 Columns: ['Energy Prices', 'Unnamed: 1']

📊 Processing file 3/3: E:\Github projects\Energy_Quant\DK1_2025.xlsx
   ✅ Loaded successfully!
   📊 Shape: (4949, 2)
   📋 Columns: ['Energy Prices', 'Unnamed: 1']

✅ Loaded 3 files successfully!
